# Assignment 5.  
## Queries combining multiple tables

In this assigment, we will fetch data from a relational database server directly into Python variables. Using these new skills, we will complete the problems from the "Problems for you to Solve" in Chapters 8, 9, and 11 in *SQL Queries for Mere Mortals*.

In [1]:
import json
import pymysql 

pymysql.install_as_MySQLdb()

with open('cred.json') as f:
    creds = json.load(f)

connection_string = "mysql://{user}:{password}@{host}".format(**creds)

In [2]:
%load_ext sql
%config SqlMagic.autocommit=True
%sql $connection_string

'Connected: dimitri@None'

### Increasing complexity of queries